Reading classics [Deep Learning Models](https://github.com/rasbt/deeplearning-models)

## Code Modules

In [ ]:
import numpy as np,pandas as pd,pylab as pl
import h5py,torch,tensorflow as tf
from torchvision.datasets import MNIST as tmnist
from torchvision import transforms
from torch.utils.data import DataLoader as tdl
from torch.utils.data import Dataset as tds
import torch.nn.functional as tnnf
from IPython.core.magic import register_line_magic
dev=torch.device("cuda:0" \
if torch.cuda.is_available() else "cpu")

## Data

In [ ]:
class tdata(tds):
    def __init__(self,X,y):   
        self.X=torch.tensor(X,dtype=torch.float32)
        self.y=torch.tensor(y,dtype=torch.int32)
    def __getitem__(self,index):
        image,label=self.X[index],self.y[index]
        return image,label
    def __len__(self):
        return self.y.shape[0]
def tloaders(x_train,y_train,x_test,y_test,batch_size):
    train=tdata(x_train,y_train)
    test=tdata(x_test,y_test)
    train_loader=tdl(dataset=train,
                     batch_size=batch_size,shuffle=True)
    test_loader=tdl(dataset=test,
                    batch_size=batch_size,shuffle=False)
    return train_loader,test_loader

In [ ]:
(x_train,y_train),(x_test,y_test)=\
tf.keras.datasets.mnist.load_data()
x_train=np.array(x_train,dtype='float32')/255
x_train=x_train.reshape(-1,28,28,1)
x_test=x_test.reshape(-1,28,28,1)
x_test=np.array(x_test,dtype='float32')/255
y_train=np.array(y_train,dtype='int32')
y_test=np.array(y_test,dtype='int32')
x_train.shape,y_train.shape

In [ ]:
train_loader,test_loader=\
tloaders(x_train,y_train,x_test,y_test,64)
for images,labels in train_loader:  
    print('Image dimensions: %s'%str(images.shape))
    print('Label dimensions: %s'%str(labels.shape))
    break

In [ ]:
fpath='../input/classification-of-handwritten-letters/'
f='LetterColorImages_123.h5'
f=h5py.File(fpath+f,'r')
keys=list(f.keys()); print(keys)
x=np.array(f[keys[1]],dtype='float32')/255
y=np.array(f[keys[2]],dtype='int32')-1
N=len(y); n=int(.2*N)
shuffle_ids=np.arange(N)
np.random.RandomState(23).shuffle(shuffle_ids)
x,y=x[shuffle_ids],y[shuffle_ids]
x_test,x_train=x[:n],x[n:]
y_test,y_train=y[:n],y[n:]
x_train.shape,y_train.shape

In [ ]:
train_loader2,test_loader2=\
tloaders(x_train,y_train,x_test,y_test,64)
for images,labels in train_loader2:  
    print('Image dimensions: %s'%str(images.shape))
    print('Label dimensions: %s'%str(labels.shape))
    break

## MLP with Sigmoid Activation & MSE Loss

In [ ]:
class MLPS():
    def __init__(self,num_features,hidden,num_classes):
        super(MLPS,self).__init__()
        self.num_features=num_features
        self.num_classes=num_classes
        self.weight1=torch.zeros(hidden,num_features, 
                                 dtype=torch.float).normal_(0.,.1)
        self.bias1=torch.zeros(hidden,dtype=torch.float)
        self.weightout=torch.zeros(self.num_classes,hidden, 
                                   dtype=torch.float).normal_(0.,.1)
        self.biasout=torch.zeros(self.num_classes,dtype=torch.float)        
    def forward(self,x):
        z1=torch.mm(x,self.weight1.t())+self.bias1
        a1=torch.sigmoid(z1)
        z2=torch.mm(a1,self.weightout.t())+self.biasout
        a2=torch.sigmoid(z2)
        return a1,a2
    def backward(self,x,a1,a2,y):  
        y_ohe=torch.FloatTensor(y.size(0),self.num_classes)
        y_ohe.zero_()
        y_ohe.scatter_(1,y.view(-1,1).long(),1)
        dloss_da2=2.*(a2-y_ohe)/y.size(0)
        da2_dz2=a2*(1.-a2)
        delta_out=dloss_da2*da2_dz2
        dz2__dw_out=a1
        dloss__dw_out=torch.mm(delta_out.t(),dz2__dw_out)
        dloss__db_out=torch.sum(delta_out,dim=0)
        dz2__a1=self.weightout
        dloss_a1=torch.mm(delta_out,dz2__a1)
        da1__dz1=a1*(1.-a1)
        dz1__dw1=x
        dloss_dw1=torch.mm((dloss_a1*da1__dz1).t(),dz1__dw1)
        dloss_db1=torch.sum((dloss_a1*da1__dz1),dim=0)
        return dloss__dw_out,dloss__db_out,dloss_dw1,dloss_db1

In [ ]:
def ohe(y,num_classes):
    y_ohe=torch.FloatTensor(y.size(0),num_classes)
    y_ohe.zero_()
    y_ohe.scatter_(1,y.view(-1,1).long(),1).float()
    return y_ohe
def tloss(targets_ohe,probs_ohe):
    return torch.mean(torch.mean((targets_ohe-probs_ohe)**2,dim=0))
def tmse(model,data_loader):
    curr_mse=torch.zeros(model.num_classes).float()
    num_examples=0
    with torch.no_grad():
        for features,targets in data_loader:
            features=features.view(-1,model.num_features)
            logits,probs=model.forward(features)
            y_ohe=ohe(targets,model.num_classes)
            loss=torch.sum((y_ohe-probs)**2,dim=0)
            num_examples+=targets.size(0)
            curr_mse+=loss
        curr_mse=torch.mean(curr_mse/num_examples,dim=0)
        return curr_mse

In [ ]:
def model_train(model,data_loader,num_epochs,learning_rate=.1):
    minibatch_cost=[]; epoch_cost=[]  
    for e in range(num_epochs):
        for batch_idx,(features,targets) in enumerate(data_loader):            
            features=features.view(-1,model.num_features)
            a1,a2=model.forward(features)
            dloss__dw_out,dloss__db_out,dloss_dw1,dloss_db1=\
            model.backward(features,a1,a2,targets)
            model.weight1-=learning_rate*dloss_dw1
            model.bias1-=learning_rate*dloss_db1
            model.weightout-=learning_rate*dloss__dw_out
            model.biasout-=learning_rate*dloss__db_out
            curr_cost=tloss(ohe(targets,model.num_classes),a2)
            minibatch_cost.append(curr_cost)
            if not batch_idx%300:
                print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f' 
                       %(e+1,num_epochs,batch_idx,
                         len(data_loader),curr_cost))        
        curr_cost=tmse(model,data_loader)
        epoch_cost.append(curr_cost)
        print('Epoch: %03d/%03d |'%(e+1,num_epochs),end=" ")
        print('Train MSE: %.5f'%curr_cost)
    return minibatch_cost,epoch_cost
def model_acc(model,data_loader):
    correct_pred,num_examples=0,0
    with torch.no_grad():
        for features,targets in data_loader:
            features=features.view(-1,model.num_features)
            _,outputs=model.forward(features)
            predicted_labels=torch.argmax(outputs,1)
            num_examples+=targets.size(0)
            correct_pred+=(predicted_labels==targets).sum()
        return correct_pred.float()/num_examples*100 

## Model 1

In [ ]:
torch.manual_seed(23)
model=MLPS(num_features=28*28*1,
           hidden=128,num_classes=10)
minibatch_cost,epoch_cost=\
model_train(model,train_loader,
            num_epochs=50,learning_rate=.07)

In [ ]:
print('Training Accuracy: %.2f'%model_acc(model,train_loader))
print('Test Accuracy: %.2f'%model_acc(model,test_loader))

In [ ]:
for features,targets in test_loader:
    break
fig,ax=pl.subplots(1,5)
for i in range(5):
    ax[i].imshow(features[5+i].view(28,28),
                 cmap=pl.cm.bone)
_,predictions=\
model.forward(features[5:10].view(-1,model.num_features))
pred_labels=torch.argmax(predictions,dim=1)
print('Predicted labels: ',pred_labels)

## Model 2

In [ ]:
torch.manual_seed(23)
model=MLPS(num_features=32*32*3,
           hidden=512,num_classes=33)
minibatch_cost2,epoch_cost2=\
model_train(model,train_loader2,
            num_epochs=200,learning_rate=.1)

In [ ]:
print('Training Accuracy: %.2f'%model_acc(model,train_loader2))
print('Test Accuracy: %.2f'%model_acc(model,test_loader2))